In [1]:
import os
import yaml
import json
import pandas as pd
from pathlib import Path

In [2]:
# parser.add_argument('mode', type=str,
#     choices=['make-ome', 'segment-ome', 'generate-spatial-features', 'generate-region-features', 'show-channels'],
#     help='Which task mip is to execute.')


# ###################
# ## show-channels ##
# ###################

# parser.add_argument('--sep', type=str, default='\n',
#     help='Seperator between channel names. Defaults to newline character (i.e. channels are displayed on seperate lines)')

# ##############
# ## make-ome ##
# ##############
# parser.add_argument('--input-tif', type=str,
#     help='Used in make-ome mode. Either directory of stitched tif files that will be combined into a single ome.tiff file, a multichannel .tif (for original codex platform), or a .qptiff (phenocycler platform).')

# parser.add_argument('--output-filepath', type=str,
#     help='Location to write ome.tiff file')

# parser.add_argument('--platform', type=str,
#     choices=['codex', 'phenocycler', 'raw'], default='phenocycler',
#     help='Which platform produced the input images. phenocycler assumes a .qptiff from the akoya phenocycler platform. codex assumes a multichannel .tif output by the original akoya codex platform. raw will save a directory of .tifs together into a multiplex image.')

# parser.add_argument('--bbox', type=str,
#     help='If desired, bbox in to crop image with. Must be the following format: "top,bottom,left,right"')

# # #################
# # ## segment-ome ##
# # #################
# # parser.add_argument('--input-tif', type=str,
# #     help='ome.tiff file to segment')

# parser.add_argument('--output-prefix', type=str, default='output',
#     help='Output prefix to use when writing cell segmentation files. Two files will be written: *_cell_segmentation.tif and *_nuclei_segmentation.tif. Default is "output". For example, if --output-prefix is "path/to/out/directory/sample" then the two output files will be named path/to/out/directory/sample_cell_segmentation.tif and path/to/out/directory/sample_nuclei_segmentation.tif.')

# parser.add_argument('--split-size', type=int, default=25000,
#     help='If image width or height is larger than --split-size, then image will be split into multiple pieces for segmentation and stitched back together. Decrease if running into memory issues.')

# parser.add_argument('--nuclei-channels', type=str, default='DAPI',
#     help='List of nuclei markers to use during segmentation. Must be the following format: "MARKER_NAME,MARKER_NAME,....". Default is "DAPI".')

# parser.add_argument('--membrane-channels', type=str, default='Pan-Cytokeratin,E-cadherin,CD45,CD8,CD3e,Vimentin,SMA,CD31,C20',
#     help='List of markers to use during membrane segmentation. Must be the following format: "MARKER_NAME,MARKER_NAME,....". Default is "Pan-Cytokeratin,E-cadherin,CD45,CD8,CD3e,Vimentin,SMA,CD31,CD20". Note that image marker names are automatically converted using mip.utils.R_CHANNEL_MAPPING')

# ###############################
# ## generate-spatial-features ##
# ###############################
# # parser.add_argument('--input-tif', type=str,
# #     help='ome.tiff file to generate spatial features for')

# # parser.add_argument('--output-prefix', type=str, default='output',
# #     help='Output prefix to use when writing output files. Two files will be written: *_spatial_features.h5ad and *_spatial_features.txt. Default is "output". For example, if --output-prefix is "path/to/out/directory/sample" then the two output files will be named path/to/out/directory/sample_spatial_features.h5ad and path/to/out/directory/sample_spatial_features.txt.')

# parser.add_argument('--labeled-image', type=str,
#     help='Filepath of labeled cell segmentation tif.')

# parser.add_argument('--thresholds', type=str,
#     help='Filepath to tab-seperated .txt file containing marker threshold values. The file should have two columns in the following order: <marker>\t<theshold>. Do not include column names/headers in the file.')

# ###############################
# ## generate-region-features ##
# ###############################
# # parser.add_argument('--input-tif', type=str,
# #     help='ome.tiff file to generate spatial features for')

# # parser.add_argument('--output-prefix', type=str, default='output',
# #     help='Output prefix to use when writing output files. Two files will be written: *_spatial_features.h5ad and *_spatial_features.txt. Default is "output". For example, if --output-prefix is "path/to/out/directory/sample" then the two output files will be named path/to/out/directory/sample_spatial_features.h5ad and path/to/out/directory/sample_spatial_features.txt.')

# parser.add_argument('--mask-tif', type=str,
#     help='Filepath of region mask tif.')

# parser.add_argument('--mask-markers', type=str, default='Pan-Cytokeratin,E-cadherin',
#     help='If --mask-tif is not provided, these markers will be used to generate a region mask.')

# parser.add_argument('--spatial-features', type=str,
#     help='Filepath to .h5ad file output by generate-spatial-features.')

In [3]:
# parser.add_argument(
#     '--username', type=str, required=True,
#     help='Omero username.')

# parser.add_argument(
#     '--password', type=str, required=True,
#     help='Omero password.')

# parser.add_argument(
#     '--group', type=str, default='HTAN',
#     help='Omero group to import or download with.')

# parser.add_argument(
#     '--project', type=str, default='Multiplex_Imaging',
#     help='Omero project to import or download with.')

# parser.add_argument(
#     '--host', type=str, default='htan-imaging.wucon.wustl.edu',
#     help='Omero host url.')

# parser.add_argument(
#     '--port', type=str, default='4064',
#     help='Omero port.')

# parser.add_argument(
#     '--filepath', type=str,
#     help='File to upload or location to save file.')

# parser.add_argument(
#     '--image-id', type=str,
#     help='ID of image to download')

# parser.add_argument(
#     '--dataset', type=str,
#     help='Omero dataset to upload image to or download image from.')

# parser.add_argument(
#     '--image-name', type=str,
#     help='Name of image in dataset to download')

In [4]:
cwl_dir = '/diskmnt/Projects/Users/estorrs/multiplex-imaging-pipeline/cwl'
Path(cwl_dir).mkdir(parents=True, exist_ok=True)

In [5]:
template = {
    'specimen_id': 'a_string',
    
    # ome generation related
    'input_tif': {
        'class': 'File',
        'path': 'a/file/path',
    },
    'platform': 'a_string',
    'bbox': 'a_string',
    
    # segmentation
    'nuclei_channels': 'a_string',
    'membrane_channels': 'a_string',
    
    # region features
    'mask_markers': 'a_string',
    
    # omero related
    'group': 'a_string',
    'project': 'a_string',
    'host': 'a_string',
    'port': 'a_string',
}

In [6]:
yaml.safe_dump(template, open(os.path.join(cwl_dir, 'template.full_imaging_workflow.yaml'), 'w'))

In [7]:
cwl = {
    'cwlVersion': 'v1.0',
    'class': 'Workflow',
    'id': 'full_imaging_workflow',
    'label': 'full_imaging_workflow',
    'inputs': [
        {
            'id': 'specimen_id',
            'type': 'string'
        },
        
        # image related
        {
            'id': 'input_tif',
            'type': 'File'
        },
        {
            'id': 'platform',
            'type': 'string?',
            'default': 'phenocycler'
        },
        {
            'id': 'bbox',
            'type': 'string?',
        },
        {
            'id': 'nuclei_channels',
            'type': 'string?',
            'default': 'DAPI'
        },
        {
            'id': 'membrane_channels',
            'type': 'string?',
            'default': 'Pan-Cytokeratin,E-cadherin,CD45,CD8,CD3e,CD4,Vimentin,SMA,CD31'
        },
        {
            'id': 'mask_markers',
            'type': 'string?',
            'default': 'Pan-Cytokeratin,E-cadherin'
        },
        
        # omero related
        {
            'id': 'project',
            'type': 'string?',
            'default': 'Multiplex_Imaging'
        },
        {
            'id': 'host',
            'type': 'string?',
            'default': 'htan-imaging.wucon.wustl.edu'
        },
        {
            'id': 'port',
            'type': 'string?',
            'default': '4064'
        },
        {
            'id': 'group',
            'type': 'string?',
            'default': 'HTAN'
        },
    ],
    'outputs': [
        {
            'id': 'ome_tiff',
            'type': 'File',
            'outputSource': 'image_preprocessing/ome_tiff'
        },
        {
            'id': 'labeled_nuclei',
            'type': 'File',
            'outputSource': 'segment_ome/labeled_nuclei'
        },
        {
            'id': 'labeled_cells',
            'type': 'File',
            'outputSource': 'segment_ome/labeled_cells'
        },
        {
            'id': 'spatial_features_txt',
            'type': 'File',
            'outputSource': 'generate_spatial_features/output_txt'
        },
        {
            'id': 'spatial_features_h5ad',
            'type': 'File',
            'outputSource': 'generate_spatial_features/output_h5ad'
        },
        {
            'id': 'region_features_txt',
            'type': 'File',
            'outputSource': 'generate_region_features/output_txt'
        },
        {
            'id': 'region_mask',
            'type': 'File',
            'outputSource': 'generate_region_features/region_mask'
        },
    ],
    'steps': [
        {
            'id': 'image_preprocessing',
            'label': 'image_preprocessing',
            'run': 'image_preprocessing_workflow.cwl',
            'in': [
                {'id': 'specimen_id', 'source': 'specimen_id'},
                {'id': 'input_tif', 'source': 'input_tif'},
                {'id': 'platform', 'source': 'platform'},
                {'id': 'bbox', 'source': 'bbox'},
                {'id': 'group', 'source': 'group'},
                {'id': 'project', 'source': 'project'},
                {'id': 'port', 'source': 'port'},
                {'id': 'host', 'source': 'host'},
            ],
            'out': [
                {'id': 'ome_tiff'}
            ]
        },
        {
            'id': 'image_processing_to_thresholds',
            'label': 'image_processing_to_thresholds',
            'run': 'image_processing_no_thresholds_workflow.cwl',
            'in': [
                {'id': 'specimen_id', 'source': 'specimen_id'},
                {'id': 'ome_tiff', 'source': 'image_preprocessing/ome_tiff'}
                {'id': 'nuclei_channels', 'source': 'nuclei_channels'},
                {'id': 'membrane_channels', 'source': 'membrane_channels'},
                {'id': 'mask_markers', 'source': 'mask_markers'},
                {'id': 'group', 'source': 'group'},
                {'id': 'project', 'source': 'project'},
                {'id': 'port', 'source': 'port'},
                {'id': 'host', 'source': 'host'},
            ],
            'out': [
                {'id': 'labeled_nuclei'},
                {'id': 'labeled_cells'},
                {'id': 'spatial_features_txt'},
                {'id': 'spatial_features_h5ad'},
                {'id': 'region_features_txt'},
                {'id': 'labeled_regions'},
                {'id': ''},
                {'id': ''},
            ]
        },
        
    ],
   'requirements': [
   ]
}

In [8]:
yaml.safe_dump(cwl, open(os.path.join(cwl_dir, 'full_imaging_workflow.cwl'), 'w'))

In [ ]:
{
            'id': 'labeled_nuclei',
            'type': 'File',
            'outputSource': 'segment_ome/labeled_nuclei'
        },
        {
            'id': 'labeled_cells',
            'type': 'File',
            'outputSource': 'segment_ome/labeled_cells'
        },
        {
            'id': 'spatial_features_txt',
            'type': 'File',
            'outputSource': 'generate_spatial_features/output_txt'
        },
        {
            'id': 'spatial_features_h5ad',
            'type': 'File',
            'outputSource': 'generate_spatial_features/output_h5ad'
        },
        {
            'id': 'region_features_txt',
            'type': 'File',
            'outputSource': 'generate_region_features/output_txt'
        },
        {
            'id': 'labeled_regions',
            'type': 'File',
            'outputSource': 'generate_region_features/region_mask'
        },